# TMSD - Ex02 - `[escreva seu nome aqui]` - `[escreva seu RA aqui]`

Bem vindo!
Neste exercício computacional você fará uma descrição e simulação do sistema original não linear ($\mathcal{M}_0$, chamaremos de "processo" ou "planta") e também do modelo linearizado ($\mathcal{M}_1$) que será obtido a partir das equações do sistema.

**Instruções:**
- Use a versão Python 3.x
- Evite sempre usar usar laços `for` e `while`, fazer contas no formato vetorial é sempre mais rápido.
- Não apague os comentários que já existem nas células de código. Mas é claro que você pode adicionar outros comentários.

**Objetivos**
- Simular o processo $\mathcal{M}_0$ não linear
- Obter o modelo $\mathcal{M}_1$, fazendo a linearização em torno de um ponto de operação
- Simular a resposta ao degrau de $\mathcal{M}_0$ e $\mathcal{M}_1$ e comparar
- Analisar os resultados

## Instruções gerais

Seu código deve estar entre os comentários `### SEU CÓDIGO COMEÇA AQUI ###` e `### FIM DO CÓDIGO ###`. Em alguns trechos será especificado "(≈ X linhas de código)" nos comentários para que você tenha uma ideia sobre o tamanho do código a ser desenvolvido naquele trecho. Lembrando que é só uma estimativa, o seu código pode ficar maior ou menor do que o especificado.

**Alguns atalhos úteis *no código*:**
- `Ctrl+Enter`: executa a célula e mantém o cursor na mesma célula
- `Shift+Enter`: executa a célula e move o cursor para a próxima célula
- `Ctrl+/`: comenta a linha de código
- `Shift+Tab`: quando o cursor estiver em uma função, mostra um HELP da função

**Alguns atalhos úteis *na célula*:**
- Cria nova célula `a`: acima, `b`: abaixo da céula selecionada
- `d` (2x): deleta célula selecionada
- `m`: define célula como texto (Markdown)
- `y`: define célula como código (Python)
- `l`: mostra numeração das linhas na célula de código
- `c`: copiar, `v`: colar, `x`: recortar célula selecionada
- `ctrl+shift+p`: mostra busca para todos comandos de célula

## Simulando o Processo Não-linear

Uma equação no espaço de estados tem o seguinte formato.

\begin{eqnarray}
\dot{\vec{x}} = f(\vec{x}, u), \\
y = h(\vec{x}, u),
\end{eqnarray}
em que $\vec{x}\in\mathbb{R}^n$ é o vetor de estados e $u$ a entrada.

### Equação da Planta

<mark>**Fazer:** escreva abaixo a equação do seu processo original, no formato de espaço de estados. Informe em seguida os valores dos parâmetros e o ponto de operação.</mark>

\begin{eqnarray}
  \dot{h}_1 & = & \frac{1}{A_{1}}\left(q_{in}-C_{1}\sqrt{h_{1}}\right)=f_{1}(h_{1},h_{2},q_{in}) \\
  \dot{h}_2 & = & \frac{1}{A_{2}}\left(q_{d}+C_{1}\sqrt{h_{1}}-C_{2}\sqrt{h_{2}}\right)=f_{2}(h_{1},h_{2},q_{in}) \\
  y & = & h_2
\end{eqnarray}
sendo os parâmetros $A_1 = 7$, $A_2 = 9$, $C_1 = 2$, $C_2 = 3$, a entrada de perturbação $q_{d} \approx 0$ e o ponto de operação considerado em $\bar{q}_{in}= 50$ l/min.

`<essas equações acima são só exemplos! Substitua pela SUA equação e APAGUE este comentário!>`

<mark>**Fazer:** implemente as equações do sistema.</mark>
1. Escreva uma função de entrada `u(t)` que implemente o sinal degrau atrasado (escolha um tempo adequado, e.g. 10 segundos), considerando que `t` pode ser um vetor;
1. Escreva a função de saída do sistema `h(x)` como função dos estados. Considere que `x` pode ser uma matriz representando a evolução dos estados ao longo do tempo;
1. Implemente a função `m0(t, x)` que a partir do vetor de estados $\vec{x}$ calcula os valores de $f(\vec{x}, u)$, que é a derivada temporal dos estados. Onde aparecer a entrada $u(t)$ na equação, chame a função entrada criada anteriormente;
1. *Não se esqueça de testar todas as funções criadas para ver se estão funcionando!!!*

In [5]:
### SEU CÓDIGO COMEÇA AQUI ### (≈ 1 linha de código)

# Não se esqueça de importar as bibliotecas!!!

# função de entrada
def u(t):
    return None

# função de saída
def h(x):
    return None

# função com a equação diferencial do sistema (note abaixo que os parâmetros são argumentos opcionais!)
def m0(t, x, A1=7, A2=9, C1=2, C2=3):
    return None

# teste das funções criadas
None

### FIM DO CÓDIGO ###

Para simularmos o sistema usaremos a função abaixo, que implementa o algoritmo de Runge-Kutta 4a ordem.
<mark>**Fazer:** apenas rode a célula abaixo.</mark> Se não rodar, comente a linha que tem o comando `@jit(nopython=True)`.

In [6]:
# Integração numérica (eficiente)
def intRK2(odeFun, x0, t, args):
    """
    Runge-Kutta integration (same parameters as scipy.integrate.odeint)
    
    Inputs:
       - odeFun: (callable) ordinary differential equation, e.g. "x, t: odeFun(x, t)"
       - x0: vector of initial states, ex. "x0 = np.array([1.0, 0.3, 3.1])" for a 3D system
       - t: np.array containing the time stamps, ex "t=np.arange(0, 100, 0.02)"
    
    Outputs:
       - x: array of states, shape (len(t), len(x0))
       - t: array of time stamps, shape (len(t))
    
    MACSIN - v001 - Leandro Freitas (dez-2017)
    """
    
    # pre-allocate state vector
    x = np.empty((len(t),len(x0))) #(len(t), len(x0))
    
    @jit(nopython=True)
    def auxFunc(x0, t, x, args):
        # initial state
        x[0, :] = x0

        # integration step
        dt = t[1]-t[0]

        # loop to compute the states
        for k in range(1, len(t)):
            k1F = dt*odeFun(x[k-1, :], t[k], *args)
            k2F = dt*odeFun(x[k-1, :] + k1F/2., t[k], *args)
            k3F = dt*odeFun(x[k-1, :] + k2F/2., t[k], *args)
            k4F = dt*odeFun(x[k-1, :] + k3F, t[k], *args)
            # compute the actual state
            x[k, :] = x[k-1, :] + (k1F+2.*k2F+2.*k3F+k4F)/6.

        return x
    
    return auxFunc(x0, t, x, args)

<mark>**Fazer:** agora vamos simular o sistema original para uma entrada em degrau.</mark>
1. Defina um vetor tempo `t` de simulação usando `np.arange(0, N*T, N)`, em que `N` é o número de pontos e `T` o passo de integração;
1. Use a função `intRK2(odeFun, x0, t, args)` definida anteriormente para simular o sistema, em que:
   - em `odeFun` você deve passar apenas o nome da função `m0` (que implementa a ODE do sistema);
   - em `x0` você deve passar o vetor de estados iniciais (pode ser um vetor nulo ou no ponto de operação, se você conhecê-lo);
   - em `t` o vetor tempo;
   - em `args` não é necessário passar nada. Eventualmente, esse argumento pode ser usado para simular o sistema com parâmetros diferentes dos originais;
   - a função retorna os estados iniciais `x0`, vetor tempo `t`, evolução dos estados ao longo do tempo `x` (e `args` se este for utilizado);
1. Use `intRK2`, simule o sistema original (integração numérica) para entrada em degrau atrasado de 10 segundos;
1. Após fazer a integração, calcule a saída do sistema usando a função `h(x)` criada anteriormente;
1. Faça um gráfico apresentando a saída `y0_1` resposta ao degrau do sistema, próximo a um ponto de operação;
1. Faça um segundo gráfico apresentando a saída `y0_2` resposta ao degrau do sistema, longe do ponto de operação;
1. No tempo antes de acontecer o degrau da entrada, o sistema deve estar em equilíbrio! Ajuste o tempo do degrau para que o sistema esteja em equilíbrio no momento do degrau.

***Obs.**: Todo gráfico deve ter rótulo nos eixos. Ajuste as escalas e eixos dos gráficos de forma a ficarem bem apresentados!!!*

***Obs.2**: uma função similar à `intRK2` está implementada na biblioteca Scipy, com nome `sp.integrate.odeint`, que é utilizada do mesmo jeito, com os mesmos argumentos*

In [7]:
### SEU CÓDIGO COMEÇA AQUI ###

# parâmetros de simulação
T = None
N = None
# vetor tempo
t = None

# simulação do sistema com degrau próximo do ponto de operação
y0_1 = None

# simulação do sistema com degrau longe do ponto de operação
y0_2 = None

### FIM DO CÓDIGO ###

**Saída esperada**:

- Gráfico da saída do sistema para um degrau próximo do ponto de operação
- Gráfico da saída do sistema para um degrau afastado do ponto de operação considerado
___

### Modelo linearizado

Consideraremos um ponto de equilíbrio como ponto de operação, denotado por $(\bar{q}_{in},\bar{h_1},\bar{h_2})$. Para um ponto de equilíbrio assintoticamente estável, se mantivermos o valor da entrada constante em $q_{in}(t)=\bar{q}_{in}$ por um tempo suficientemente longo, observaremos os valores dos estados $h_1(t)=\bar{h_1}$ e $h_2(t)=\bar{h_2}$ em estado estacionário. Note que, se os estados e as entradas ficam constantes, a saída também chegará no seu valor de equilíbrio $y(t)=\bar{y}$.

Para obter um modelo linear, podemos utilizar a expansão em série de Taylor em torno do ponto de equilíbrio $(\bar{h_1}, \bar{h_2}, \bar{q}_{in})$. Primeiro definiremos as variáveis de "desvio", que são as variáveis de estado e a saída deslocados.

#### Variáveis de desvio

Estados

\begin{eqnarray*}
x_{1} & = & h_{1}-\bar{h}_{1}\\
x_{2} & = & h_{2}-\bar{h}_{2}
\end{eqnarray*}

Entrada de controle
$$u=q_{in}-\bar{q}_{in}$$

Entrada de perturbação
$$d=q_{d}-\bar{q}_{d}$$

Saída medida
$$y=h_{2}-\bar{h}_{2}$$


Usando a expansão em série de Taylor, em torno do ponto de equilíbrio, obtemos:

\begin{eqnarray*}
\frac{dx_{1}}{dt} & = & f_{1}(\bar{h}_{1}+x_{1},\bar{h}_{2}+x_{2},\bar{q}_{in}+u,\bar{q}_{d}+d)\\
 & \approx & \underbrace{f_{1}(\bar{h}_{1},\bar{h}_{2},\bar{q}_{in})}_{0}+\left.\frac{\partial f_{1}}{\partial h_{1}}\right|_{SS}x_{1}+\left.\frac{\partial f_{1}}{\partial h_{2}}\right|_{SS}x_{2}+\left.\frac{\partial f_{1}}{\partial q_{in}}\right|_{SS}u+\left.\frac{\partial f_{1}}{\partial q_{d}}\right|_{SS}d\\
 & \approx & \left(-\frac{C_{1}}{2A_{1}\sqrt{\bar{h}_{1}}}\right)x_{1}+\left(0\right)x_{2}+\left(\frac{1}{A_{1}}\right)u+\left(0\right)d\\
 & \approx & \left(-\frac{C_{1}^{2}}{2A_{1}\bar{q}_{in}}\right)x_{1}+\left(\frac{1}{A_{1}}\right)u ,
\end{eqnarray*}

para a primeira equação de estado e para a segunda:

\begin{eqnarray*}
\frac{dx_{2}}{dt} & = & f_{2}(\bar{h}_{1}+x_{1},\bar{h}_{2}+x_{2},\bar{q}_{in}+u,\bar{q}_{d}+d)\\
 & \approx & \underbrace{f_{2}(\bar{h}_{1},\bar{h}_{2},\bar{q}_{in})}_{0}+\left.\frac{\partial f_{2}}{\partial h_{1}}\right|_{SS}x_{1}+\left.\frac{\partial f_{2}}{\partial h_{2}}\right|_{SS}x_{2}+\left.\frac{\partial f_{2}}{\partial q_{in}}\right|_{SS}u+\left.\frac{\partial f_{2}}{\partial q_{d}}\right|_{SS}d\\
 & \approx & \left(\frac{C_{1}}{2A_{2}\sqrt{\bar{h}_{1}}}\right)x_{1}+\left(-\frac{C_{2}}{2A_{2}\sqrt{\bar{h}_{2}}}\right)x_{2}+\left(0\right)u+\left(\frac{1}{A_{2}}\right)d\\
 & \approx & \left(\frac{C_{1}^{2}}{2A_{2}\bar{q}_{in}}\right)x_{1}+\left(-\frac{C_{2}^{2}}{2A_{2}\bar{q}_{in}}\right)x_{2}+\left(\frac{1}{A_{2}}\right)d .
\end{eqnarray*}

A saída medida fica:

\begin{eqnarray*}
y & = & h_{2}-\bar{h}_{2}\\
 & = & x_{2}
\end{eqnarray*}

<mark>**Fazer:** apague as equações acima e coloque no lugar o seu modelo linearizado em torno do ponto de operação considerado. Não precisa colocar o desenvolvimento das equações, apenas o modelo final. </mark>

<mark>**Fazer:** simule o modelo linearizado na célula abaixo.</mark>
1. Crie uma função `m1` que implemente a equação diferencial do modelo linearizado
2. Simule o modelo original `m0` e o linearizado `m1` com entrada em degrau próximo do ponto de operação
1. Mostre um gráfico comparando as duas saídas `y0_1` e `y1_1`
2. Simule o modelo original `m0` e o linearizado `m1` com entrada em degrau afastado do ponto de operação
1. Mostre um gráfico comparando as duas saídas `y0_2` e `y1_2`

In [8]:
### SEU CÓDIGO COMEÇA AQUI ###
None
### FIM DO CÓDIGO ###

**Saída esperada**:

- Gráfico da saída do processo $\mathcal{M}_0$ e do modelo $\mathcal{M}_1$ para um degrau próximo do ponto de operação
- Gráfico da saída do processo $\mathcal{M}_0$ e do modelo $\mathcal{M}_1$ para um degrau afastado do ponto de operação considerado
___

## Conclusões

<mark>**Fazer:** escreva aqui as principais conclusões do trabalho.</mark>

___
Fim (ufa!)